# Voronoi Image
Demo the Voronoi image code

In [ ]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import skimage.io
import time
import torch
import glob

import sys
sys.path.append("../src")
from imtools import get_voronoi_image
from tsp import *

In [ ]:
## Parameters
device = 'cpu'
n_points = 3000
n_iters = 100
#I = skimage.io.imread("../data/images/layla.png")
I = skimage.io.imread("../data/imagenet/n01496331/n01496331_11242.JPEG")
N = min(I.shape[0], I.shape[1])
I = I[0:N, 0:N, ...]
plt.imshow(I)
print("Compression ratio", I.shape[0]*I.shape[1]/n_points)

In [ ]:
tic = time.time()
J, X, final_cost = get_voronoi_image(I, device, n_points, n_neighbs=2, n_iters=n_iters, verbose=False, plot_iter_interval=0, use_lsqr=False)
print("X.shape", X.shape)
rmse = np.sqrt(final_cost/I.size)
print("Elapsed Time: {:.3f}".format(time.time()-tic))
plt.imshow(J)
plt.title("Final cost {:.3f}, RMSE: {:.3f}".format(final_cost, rmse*255))
plt.savefig("Final.png", bbox_inches='tight')

In [ ]:
tic = time.time()
Y = get_tsp_tour(X)
print("Elapsed time: {:.3f}".format(time.time()-tic))

In [ ]:
from scipy.spatial import KDTree

M = I.shape[0]
N = I.shape[1]
xpix, ypix = np.meshgrid(np.linspace(0, 1, N), np.linspace(0, 1, M), indexing='xy')
xpix = xpix.flatten()
ypix = ypix.flatten()
X = np.array([xpix, ypix]).T

gridtree = KDTree(X)

plt.figure(figsize=(8, 8))
plot_idx = 0
for i in range(Y.shape[0] + 500):
    plot_point = True
    if i >= Y.shape[0]:
        plot_point = False
    plt.clf()
    i = min(i, Y.shape[0]-1)
    tree = KDTree(Y[0:i+1, 0:2])
    _, idx = tree.query(X, k=1) # Nearest landmark indices to pixels
    idx = idx.flatten()
    J = Y[idx, 2::]
    J = np.reshape(J, I.shape)
    plt.imshow(J)
    # Nearest pixel to point at index
    if plot_point:
        _, idx = gridtree.query([Y[i, 0:2]])
        idx = idx.flatten()[0]
        plt.scatter(xpix[idx]*N, ypix[idx]*M)
    plt.savefig("TSP{}.png".format(plot_idx))
    plot_idx += 1